## Hardware Details
[GCP](https://cloud.google.com/) VM: [n1-highmem-16](https://cloud.google.com/compute/docs/machine-types#n1_machine_types) (16 vCPUs, 104 GB memory)

In [1]:
%%bash
lscpu

Architecture:          x86_64
CPU op-mode(s):        32-bit, 64-bit
Byte Order:            Little Endian
CPU(s):                16
On-line CPU(s) list:   0-15
Thread(s) per core:    2
Core(s) per socket:    8
Socket(s):             1
NUMA node(s):          1
Vendor ID:             GenuineIntel
CPU family:            6
Model:                 63
Model name:            Intel(R) Xeon(R) CPU @ 2.30GHz
Stepping:              0
CPU MHz:               2300.000
BogoMIPS:              4600.00
Hypervisor vendor:     KVM
Virtualization type:   full
L1d cache:             32K
L1i cache:             32K
L2 cache:              256K
L3 cache:              46080K
NUMA node0 CPU(s):     0-15
Flags:                 fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc eagerfpu pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hyperviso

In [2]:
%%bash
cat /proc/meminfo | head -n1

MemTotal:       107091244 kB


## Basic functions

In [3]:
import dask.dataframe as dd
import pandas as pd
import numpy as np
import random
import string
import gc

In [4]:
NROFCORES=16

In [5]:
def createTable(rowCount):
    gc.collect()
    return dd.from_pandas(pd.DataFrame({'bucket': [''.join(random.choices(string.ascii_lowercase, k=2)) for _ in range(rowCount)],
                  'weight': [random.uniform(0, 2) for _ in range(rowCount)],
                  'qty': [random.randint(0, 100) for _ in range(rowCount)],
                  'risk': [random.randint(0, 10) for _ in range(rowCount)]}), NROFCORES)

In [6]:
def executeQueryJoin(t):
    res = t.groupby('bucket').agg({'bucket': 'count', 'qty': [sum, np.mean], 'risk': [sum, np.mean]}).compute()
    res.columns = res.columns.map('_'.join)
    return res.rename(columns={'bucket_count':'NR', 'qty_sum':'TOTAL_QTY','qty_mean':'AVG_QTY', 
                        'risk_sum':'TOTAL_RISK','risk_mean':'AVG_RISK'}).join(
        t.groupby('bucket').apply(lambda g: np.average(g.qty, weights=g.weight), meta=('x', 'f8')).to_frame('W_AVG_QTY').compute()).join(
        t.groupby('bucket').apply(lambda g: np.average(g.risk, weights=g.weight), meta=('x', 'f8')).to_frame('W_AVG_RISK').compute())


In [7]:
def my_agg(x):
    data = {'NR': x.bucket.count(),
            'TOTAL_QTY': x.qty.sum(),
            'AVG_QTY': x.qty.mean(),
            'TOTAL_RISK': x.risk.sum(),
            'AVG_RISK': x.risk.mean(),
            'W_AVG_QTY':  np.average(x.qty, weights=x.weight),
            'W_AVG_RISK':  np.average(x.risk, weights=x.weight)
           }
    return pd.Series(data, index=['NR', 'TOTAL_QTY', 'AVG_QTY', 'TOTAL_RISK', 
                                  'AVG_RISK', 'W_AVG_QTY', 'W_AVG_RISK'])

def executeQueryApply(t):
    return t.groupby('bucket').apply(my_agg,
        meta={'NR': 'i8', 'TOTAL_QTY': 'i8', 'AVG_QTY': 'f8', 'TOTAL_RISK': 'i8', 'AVG_RISK': 'f8', 
              'W_AVG_QTY': 'f8', 'W_AVG_RISK': 'f8'}).compute().astype({
        'NR': 'int64', 'TOTAL_QTY': 'int64', 'TOTAL_RISK': 'int64'})

## Row Number 10k

In [8]:
t = createTable(10 * 1000)

In [9]:
%timeit executeQueryJoin(t)

2.13 s ± 358 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [10]:
%timeit executeQueryApply(t)

1.98 s ± 34.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Row Number 100k

In [11]:
del t
t = createTable(100 * 1000)

In [12]:
%timeit executeQueryJoin(t)

2.22 s ± 14.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [13]:
%timeit executeQueryApply(t)

2.16 s ± 18.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Row Number 1M

In [14]:
del t
t = createTable(1000 * 1000)

In [15]:
%timeit executeQueryJoin(t)

4.42 s ± 40 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [16]:
%timeit executeQueryApply(t)

3.48 s ± 27.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Row Number 10M

In [18]:
del t
t = createTable(10 * 1000 * 1000)

In [19]:
%timeit executeQueryJoin(t)

25 s ± 278 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [20]:
%timeit executeQueryApply(t)

15.3 s ± 160 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Row Number 100M

In [21]:
del t
t = createTable(100 * 1000 * 1000)

In [22]:
%timeit executeQueryJoin(t)

4min 23s ± 2.19 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [23]:
%timeit executeQueryApply(t)

2min 26s ± 704 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
